In [1]:
# -*- coding: UTF-8 -*-


In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]


In [9]:
# 返回数据集中出现的单项数据集合(候选项集)，返回类型是frozenset类型为元素的集合
def createC1(dataset):
    C1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return map(frozenset, C1)


In [17]:
# D数据集 Ck项集
# 扫描数据集，计算项集支持度 返回大于指定支持度的项集
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not ssCnt.has_key(can):
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numitems = float(len(D))  # 数据集长度
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numitems
        if support >= minSupport:
            retList.insert(0, key)
        supportData[key] = support
    return retList, supportData


In [18]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(Lk[i])[:k - 2]
            L2 = list(Lk[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j])
    return retList


In [19]:
def apriori(dataset, minsupport=0.5):
    C1 = createC1(dataset)  # 候选项集
    D = map(set, dataset)  # 数据集
    L1, supportData = scanD(D, C1, minsupport)  # 频繁项集与支持度
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [20]:
dataset=loadDataSet()
dataset

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [21]:
C1=createC1(dataset)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [22]:
D=map(set,dataset)
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [23]:
L1,suppData0=scanD(D,C1,0.5)
L1

[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})]

In [24]:
L, suppData = apriori(dataset)  #得到频繁项集 默认支持度大于0.5
L


[[frozenset({1}), frozenset({3}), frozenset({2}), frozenset({5})],
 [frozenset({1, 3}), frozenset({2, 5}), frozenset({2, 3}), frozenset({3, 5})],
 [frozenset({2, 3, 5})],
 []]

In [25]:
#计算关联规则

In [30]:
#               ss
def calConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]  #置信度
        if conf >= minConf:
            print freqSet - conseq, "--->", conseq, "conf", conf
            brl.append(conseq)
    return prunedH


In [31]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


In [32]:
def generateRules(L, supportData, minConf=0.7):
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList


In [33]:
L,suppData=apriori(dataset)


In [34]:
rules = generateRules(L, suppData)
rules


frozenset([1]) ---> frozenset([3]) conf 1.0
frozenset([5]) ---> frozenset([2]) conf 1.0
frozenset([2]) ---> frozenset([5]) conf 1.0


[frozenset({3}), frozenset({2}), frozenset({5})]

In [35]:
rules=generateRules(L,suppData,0.5)
rules

frozenset([3]) ---> frozenset([1]) conf 0.666666666667
frozenset([1]) ---> frozenset([3]) conf 1.0
frozenset([5]) ---> frozenset([2]) conf 1.0
frozenset([2]) ---> frozenset([5]) conf 1.0
frozenset([3]) ---> frozenset([2]) conf 0.666666666667
frozenset([2]) ---> frozenset([3]) conf 0.666666666667
frozenset([5]) ---> frozenset([3]) conf 0.666666666667
frozenset([3]) ---> frozenset([5]) conf 0.666666666667
frozenset([5]) ---> frozenset([2, 3]) conf 0.666666666667
frozenset([3]) ---> frozenset([2, 5]) conf 0.666666666667
frozenset([2]) ---> frozenset([3, 5]) conf 0.666666666667


[frozenset({1}),
 frozenset({3}),
 frozenset({2}),
 frozenset({5}),
 frozenset({2}),
 frozenset({3}),
 frozenset({3}),
 frozenset({5}),
 frozenset({2, 3}),
 frozenset({2, 5}),
 frozenset({3, 5})]